# ABS Quarterly Business Indicators 5676

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# local imports
from henderson import hma
from plotting import (
    calc_growth,
    clear_chart_dir,
    line_plot,
    plot_growth_finalise,
    recalibrate_series,
    set_chart_dir,
)
from abs_data_capture import (
    find_id,
    fix_abs_title,
    AbsLandingPage,
    get_abs_data,
    get_fs_constants,
    metacol,
    get_plot_constants,
)

In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# display charts
SHOW = False

## Get data from ABS

In [4]:
landing_page = AbsLandingPage(
    theme="economy",
    parent_topic="business-indicators",
    topic="business-indicators-australia",
)
abs_dict = get_abs_data(landing_page)
source, CHART_DIR, Cat_ID, meta = get_fs_constants(abs_dict, landing_page)
RECENT, plot_times, plot_tags = get_plot_constants(meta)

Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/64043caf10c33b8ae94b7c635b3a7e5d--Download-all.zip
Extracting DataFrames from the zip-file ...


In [5]:
print("Latest data: ", abs_dict["1"].index[-1])

Latest data:  2023Q3


## Plot

### Set-up some constants we will use across all plots

In [6]:
plt.style.use("fivethirtyeight")
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)

### Headline charts

In [7]:
def headline():
    """Produce headline charts."""

    series_type = "Seasonally Adjusted"
    headlines = [
        "Inventories ;  Total (State) ;  Total (Industry) ;  Chain Volume Measures ;",
        "Profit before Income Tax ;  Total (State) ;  Total (Industry) ;  Current Price ;  CORP ;",
        "Gross Operating Profits ;  Total (State) ;  Total (Industry) ;  "
        "Current Price ;  TOTAL (SCP_SCOPE) ;",
        "Wages ;  Total (State) ;  Total (Industry) ;  Current Price ;",
    ]
    recent = plot_times[1]
    for h in headlines:
        # get the data
        row = meta[
            (meta[metacol.stype] == series_type)
            & meta[metacol.did].str.contains(h, regex=False)
        ].iloc[0]
        series_id, units, table = row[metacol.id], row[metacol.id], row[metacol.table]
        series, units = recalibrate_series(abs_dict[table][series_id], units)

        # plot
        lfooter = f"Australia. {series_type.capitalize()}. "
        title, lfooter = fix_abs_title(
            f"Business indicators: {row[metacol.did]}", lfooter
        )
        line_plot(
            series,
            starts=recent,
            title=title,
            ylabel=units,
            pre_tag="!",
            rfooter=f"{source} T{table}",
            lfooter=lfooter,
            show=SHOW,
        )

        growth = calc_growth(series)
        plot_growth_finalise(
            *growth,
            from_=pd.Period(recent, freq="Q"),
            title=f"Growth in {title.title()}",
            pre_tag="!",
            rfooter=f"{source} T{table}",
            lfooter=lfooter,
            show=SHOW,
        )

In [8]:
headline()

### Profits vs wages

In [9]:
def profits_v_wages():
    """Produce provists versus wages charts."""

    series_type = "Seasonally Adjusted"

    wages_table = "17"
    wages_did = "Wages ;  Total (State) ;  Total (Industry) ;  Current Price ;"
    wages_name = wages_did.split(";", maxsplit=1)[0].strip()
    wages_id, wage_units = find_id(
        meta,
        {
            wages_table: metacol.table,
            series_type: metacol.stype,
            wages_did: metacol.did,
        },
        verbose=False,
    )
    wages = abs_dict[wages_table][wages_id].dropna()

    profits_table = "15"
    profits_did = (
        "Gross Operating Profits ;  Total (State) ;  Total (Industry) ;  "
        "Current Price ;  TOTAL (SCP_SCOPE) ;"
    )
    profits_name = profits_did.split(";", maxsplit=1)[0].strip()
    profits_id, profits_units = find_id(
        meta,
        {
            profits_table: metacol.table,
            series_type: metacol.stype,
            profits_did: metacol.did,
        },
        verbose=False,
    )
    profits = abs_dict[profits_table][profits_id].dropna()

    # plot profits as a share of wages and prices
    assert wage_units == profits_units
    rfooter = f"{source} Tables: {wages_table}:{wages_id}, {profits_table}:{profits_id}"
    profit_wage_share = profits / (profits + wages) * 100
    hma_term = 7
    trend = hma(profit_wage_share.dropna(), hma_term)
    df = pd.DataFrame(
        {
            f"{profits_name} share": profit_wage_share,
            "Henderson moving average": trend,
        }
    )

    line_plot(
        df,
        title="Profits as a share of profits plus wages",
        ylabel="Per cent",
        rfooter=rfooter,
        lfooter=f"Australia. {series_type.capitalize()} series. "
        f"{hma_term}-term Henderson moving average. ",
        show=SHOW,
    )

    # Plot values
    assert "Current Price" in wages_did and "Current Price" in profits_did
    assert "Millions" in wage_units
    df = pd.DataFrame({"Wages": (wages / 1000), "Profits": (profits / 1000)})
    line_plot(
        df,
        title="Profits vs Wages",
        ylabel="$ Billions",
        rfooter=rfooter,
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
        show=SHOW,
    )

    # plot wage and profits index (note starting point ptoblem)
    assert profits.index[0] == wages.index[0]  # common start
    profits_index = profits / profits.iloc[0] * 100
    wages_index = wages / wages.iloc[0] * 100
    df = pd.DataFrame(
        {
            f"{wages_name} index": wages_index,
            f"{profits_name} index": profits_index,
        }
    )
    line_plot(
        df,
        title="Profits index vs Wages index",
        ylabel="Index",
        rfooter=rfooter,
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
        show=SHOW,
    )

In [10]:
profits_v_wages()

### Inventories by industry sector

In [11]:
def inventories():
    """Produce Inventory charts."""

    # identify the plotable items ...
    series_type = "Seasonally Adjusted"
    measure_type = "Chain Volume Measures"
    rows = meta[
        (meta[metacol.stype] == series_type)
        & meta[metacol.did].str.contains("Inventories", regex=False)
        & meta[metacol.did].str.contains(measure_type, regex=False)
        & meta[metacol.unit].str.contains("$", regex=False)
    ]

    for _, row in rows.iterrows():
        # get the data for wacg poltable item
        series_id, units, table = row[metacol.id], row[metacol.unit], row[metacol.table]
        data = abs_dict[table]
        series, units = recalibrate_series(data[series_id], units)

        # plot the data
        lfooter = f"Australia. {series_type.capitalize()}. "
        title, lfooter = fix_abs_title(row[metacol.did], lfooter)
        line_plot(
            series,
            starts=RECENT,
            title=title,
            ylabel=units,
            rfooter=f"{source} T{table}",
            lfooter=lfooter,
            show=SHOW,
        )

In [12]:
inventories()

## Wages Growth

In [13]:
def wage_growth():
    """Produce wage growth charts."""

    series_type = "Seasonally Adjusted"
    series_did = "Wages"
    rows = meta[
        meta[metacol.did].str.contains(series_did, regex=False)
        & (meta[metacol.stype] == series_type)
        & meta[metacol.unit].str.contains("$", regex=False)
    ]

    for _, row in rows.iterrows():
        # get the data
        series_id, units, table = row[metacol.id], row[metacol.unit], row[metacol.table]
        data = abs_dict[table]
        series, units = recalibrate_series(data[series_id], units)
        lfooter = f"Australia. {series_type.capitalize()}. "
        title, lfooter = fix_abs_title(row[metacol.did], lfooter)
        growth = calc_growth(series)
        plot_growth_finalise(
            *growth,
            from_=pd.Period(RECENT, freq="Q"),
            title=f"Growth: {title}",
            rfooter=f"{source} T{table}",
            lfooter=lfooter,
            show=SHOW,
        )

In [14]:
wage_growth()

## Finished

In [15]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sun Feb 18 2024 20:49:01

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.21.0

pandas    : 2.2.0
matplotlib: 3.8.3

Watermark: 2.4.3



In [16]:
print("Finished")

Finished
